# Real Estate data cleaning and analysis

Real Estate ETL is a data engineering project focused on analyzing the real estate market in India. The project involves collecting and cleaning real estate data from various sources and aims to provide insights into property market across major cities in India. The data has been collected from the Real Estate Data from 7 Indian Cities dataset available on Kaggle. The project utilizes Extract, Transform, and Load (ETL) processes to prepare the data for analysis.

- Dataset is available on kaggle and also under the `data/raw` directory
- Project created by Aaryadev Ghosalkar

## Libraries
- Pathlib: Provides consistent API for dealing with file system paths across multiple operating systems
- Pandas: Loading cleaning and saving data
- openpyxl: Used internally by pandas to save data to Excel file

In [25]:
import pandas as pd
import pathlib


In [26]:
DATA_DIR = pathlib.Path("./data/")

In [27]:
# Read CSV "Real Estate Data V21.csv" into pandas make sure path is a pathlib path
# data\raw\Real Estate Data V21.csv
data_path = DATA_DIR / "raw" / "Real Estate Data V21.csv"
raw_df = pd.read_csv(data_path)
df = raw_df.copy()
raw_df.head()


,Name,Property Title,Price,Location,Total_Area,Price_per_SQFT,Description,Baths,Balcony
0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",₹1.99 Cr,"Kanathur Reddikuppam, Chennai",2583,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,4,Yes
1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,₹2.25 Cr,"Ramanathan Nagar, Pozhichalur,Chennai",7000,3210.0,Looking for a 10 BHK Independent House for sal...,6,Yes
2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",₹1.0 Cr,"Kasthuribai Nagar, West Tambaram,Chennai",1320,7580.0,"Property for sale in Tambaram, Chennai. This 3...",3,No
3,"Naveenilaya,Chepauk, Triplicane,Chennai",7 BHK Independent House for sale in Triplicane...,₹3.33 Cr,"Naveenilaya,Chepauk, Triplicane,Chennai",4250,7840.0,Entire Building for sale with 7 units of singl...,5,Yes
4,VGN Spring Field Phase 1,"2 BHK Flat for sale in Avadi, Chennai",₹48.0 L,"Avadi, Chennai",960,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",3,Yes


## First look at the data

Taking a first look at this data we can see some obvious things that need to be done in order to make this data usable these include the following:

- Change balcony to a Boolean with true and false
- We can split the location into 2 columns suburb and city to accommodate a more detailed analysis
- The price needs to be converted to an Integer
  - Rupees can be removed from all of these since the analysis is based on indian cities the currency is implied
  - extract the unit (Cr or L or K) *We will see later that there are some inconsistent values
  - expand the price into a number (48.0 L becomes 48,00,000)
- Split the Property title into subfields
  - Number of bedrooms (1 for Studio)..
  - Extract the property type (Flat, Independent House...)



In [28]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14528 entries, 0 to 14527
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            14528 non-null  object 
 1   Property Title  14528 non-null  object 
 2   Price           14528 non-null  object 
 3   Location        14528 non-null  object 
 4   Total_Area      14528 non-null  int64  
 5   Price_per_SQFT  14528 non-null  float64
 6   Description     14528 non-null  object 
 7   Baths           14528 non-null  int64  
 8   Balcony         14528 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 1021.6+ KB


In [29]:
df.shape

(14528, 9)

In [30]:
df.isnull().sum()

Name              0
Property Title    0
Price             0
Location          0
Total_Area        0
Price_per_SQFT    0
Description       0
Baths             0
Balcony           0
dtype: int64

## Getting the easy transformations out the way


### 1. Changing the Balcony column to a Boolean

`df[`hasBalcony`] = raw_df[`Balcony`].map({`Yes`: True, `No`: False})`

This line creates a new column named `hasBalcony`. It maps the values in the `Balcony` column where `Yes` is mapped to `True` and `No` is mapped to `False`. This operation assigns the corresponding boolean values.

### 2. Extracting the city

`df[`city`] = raw_df[`Location`].str.split(`,`).str[-1].str.strip()`

This line creates a new column named `city`. It extracts the last element after splitting the values in the `Location` column of `raw_df` by commas (`,`). The `.str[-1]` notation represents accessing the last element after the split. The `.str.strip()` function is then applied to remove any leading or trailing whitespaces. This operation assigns the extracted city names to the `city` column.

## Extracting the suburb

`df[`suburb`] = raw_df[`Location`].str.split(`,`).str[0].str.strip()`

This line creates another new column named `suburb`. It extracts the first element after splitting the values in the `Location` column of `raw_df` by commas (`,`). The `.str[0]` notation represents accessing the first element after the split. The `.str.strip()` function is then applied to remove any leading or trailing whitespaces. This operation assigns the extracted suburb names to the `suburb` column in `df`.


In [31]:
df["hasBalcony"] = raw_df["Balcony"].map({
    "Yes":True,
    "No":False
})

# TODO: Try to use a regex and the extract method to extract the city and suburb
df["city"] = raw_df["Location"].str.split(",").str[-1].str.strip()
df["suburb"] = raw_df["Location"].str.split(",").str[0].str.strip()

## Converting the price into a numeric field 

To do this we first need to extract value and unit then convert the value to a float and multiply it by a power of 10 based on the unit (Unit is denoted by `suf` for suffix in the code) however there are values in the dataset that don't fit this pattern these are values where the price has lowercase k instead of K like all the others and Lacs written in full instead of L some values did not contain a suffix (These were rare enough to analyse by hand and determine based to knowledge of properties that these had to be in Cr).

The inconsistent values were found using this:

```python

def parse_price(price: str) -> int:
    try:
        price = price.replace('₹', '')
        val, suf = price.split(' ')
        if suf == 'Cr':
            val = float(val) * 1_00_00_000
        elif suf == 'L':
            val = float(val) * 1_00_000
        elif suf == 'K':
            val = float(val) * 1_000
        return int(val)
    except:
        return None

raw_df[prices.isnull()]

```

In [44]:
# Replace the matched pattern in the "Price" column with a modified version.
# The pattern is a numeric value with two decimal places, preceded by the currency symbol "₹".
prices = raw_df["Price"]
prices = prices.str.replace(r"₹([0-9].[0-9]$)", r"₹\1 Cr", regex=True) # Replace ₹x.x with ₹x.x Cr
# - `\1` in the replacement string represents the matched numeric value within parentheses.
# - " Cr" is appended to the matched value to indicate "Crore".

# The code uses the `str.replace()` method with the `regex=True` parameter to enable regex matching.
# It applies the regex pattern to each value in the "Price" column and replaces the matched pattern
# with the modified version that includes " Cr" at the end.

# This code transforms the values in the "Price" column from a numeric format to a "Crore" format.
# For example, a value like "₹2.5" would be replaced with "₹2.5 Cr".
# The modified values indicate the price in terms of Crore (1 Crore = 10,000,000).
# This transformation facilitates consistent representation and subsequent analysis of the price data.

prices = prices.str.replace("Lacs", "L", regex=False) # Replace ₹x.x Lacs with ₹x.x L
prices = prices.str.replace("k", " K", regex=False) # Replace ₹x.x k with ₹x.x K

def parse_price(price: str) -> int:
    """
    Parses the price string and converts it into an integer value.

    Parameters:
        price (str): The price string to be parsed.

    Returns:
        int: The parsed price as an integer. Returns None if parsing fails.

    Example:
        - parse_price("₹2.5 Cr")  # Returns 250000000
        - parse_price("₹50 L")    # Returns 5000000
        - parse_price("₹10 K")    # Returns 10000

    Note:
        - The multipliers used for conversion are as follows:
            - "Cr": Crore (1 Crore = 10,000,000)
            - "L": Lakh (1 Lakh = 100,000)
            - "K": Thousand (1 Thousand = 1,000)
    """
    try:
        price = price.replace('₹', '')  # Remove currency symbol
        val, suf = price.split(' ')
        if suf == 'Cr':
            val = float(val) * 10_000_000  # 1 Crore = 10,000,000
        elif suf == 'L':
            val = float(val) * 100_000  # 1 Lakh = 100,000
        else:
            val = float(val) * 1,000  # 1 Thousand = 1,000
        return int(val)
    except:
        return None

prices = prices.map(parse_price)
prices[14491] = 55_000 # Very weird edge case had to apply a manual fix or drop it I suppose


In [41]:
df["price_parsed"] = prices

In [43]:
# Extract specific information from the "Property Title" column using regex pattern matching.
p_type = raw_df["Property Title"].str.extract(r"([0-9]+)+\s+(BHK|RK|BH|R)\s+(.*?)\s+for sale")
# - `extract()` is applied to the "Property Title" column of the DataFrame `raw_df`.
# - The regex pattern is used to identify and capture the desired information within the column.

# The regex pattern matches a common structure in property titles that typically include:
# - A numeric value indicating the number of bedrooms or rooms (e.g., 2 BHK, 1 RK).
# - A code or abbreviation representing the type of property (e.g., BHK, RK, BH, R).
# - The name or description of the property.
# - The string " for sale" indicating the purpose of the listing.

# By using capturing groups in the regex pattern, the code extracts the following information:
# - The numeric value indicating the number of bedrooms or rooms.
# - The property type code or abbreviation.
# - The name or description of the property.

# The extracted information will be returned as a new DataFrame with separate columns for each
# captured group. The columns will contain the extracted values corresponding to the regex pattern.

# TODO: Bedrooms isn't the correct thing to call this but unsure what's a good name
p_type.columns = ["Bed Rooms", "Room Type", "Property Type"]

p_type["Bed Rooms"] = p_type["Bed Rooms"].fillna(1)                # There has to be 1 Room at least
p_type["Property Type"] = p_type["Property Type"].fillna("Studio") # These are studio flats so no rooms
p_type["Room Type"] = p_type["Room Type"].fillna("R")              # These are studio flats"

p_type

,Bed Rooms,Room Type,Property Type
0,4,BHK,Flat
1,10,BHK,Independent House
2,3,BHK,Flat
3,7,BHK,Independent House
4,2,BHK,Flat
...,...,...,...
14523,2,BHK,Flat
14524,1,BHK,Independent House
14525,1,BHK,Flat
14526,3,BHK,Flat


## Combine everything to form a Final data frame

This creates the final data frame which will eventually be saved into the Excel file, in this data frame we have all the transformed and mined columns as well as the raw values in the columns in the cod below we will be doing the following:

- Removing the unused columns:
  - `Property Title`:  We have extracted information from this including number of room, Room type and property type
  - `Price`: Converted to Integer
  - `Location`: Converted to suburb and city column
  - `Balcony`: Converted to boolean
- And change the data types to something much more efficient

In [54]:
final = pd.concat([df, p_type], axis=1) # Combine the temporary dataframe to create a final data frame that can be saved
final

,Name,Property Title,Price,Location,Total_Area,Price_per_SQFT,Description,Baths,Balcony,hasBalcony,city,suburb,price_parsed,Bed Rooms,Room Type,Property Type
0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",₹1.99 Cr,"Kanathur Reddikuppam, Chennai",2583,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,4,Yes,True,Chennai,Kanathur Reddikuppam,19900000.0,4,BHK,Flat
1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,₹2.25 Cr,"Ramanathan Nagar, Pozhichalur,Chennai",7000,3210.0,Looking for a 10 BHK Independent House for sal...,6,Yes,True,Chennai,Ramanathan Nagar,22500000.0,10,BHK,Independent House
2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",₹1.0 Cr,"Kasthuribai Nagar, West Tambaram,Chennai",1320,7580.0,"Property for sale in Tambaram, Chennai. This 3...",3,No,False,Chennai,Kasthuribai Nagar,10000000.0,3,BHK,Flat
3,"Naveenilaya,Chepauk, Triplicane,Chennai",7 BHK Independent House for sale in Triplicane...,₹3.33 Cr,"Naveenilaya,Chepauk, Triplicane,Chennai",4250,7840.0,Entire Building for sale with 7 units of singl...,5,Yes,True,Chennai,Naveenilaya,33300000.0,7,BHK,Independent House
4,VGN Spring Field Phase 1,"2 BHK Flat for sale in Avadi, Chennai",₹48.0 L,"Avadi, Chennai",960,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",3,Yes,True,Chennai,Avadi,4800000.0,2,BHK,Flat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14523,"Krishna Park Extension, Tilak Nagar,New Delhi","2 BHK Flat for sale in Tilak Nagar, New Delhi",₹40.0 L,"Krishna Park Extension, Tilak Nagar,New Delhi",1053,3800.0,Looking for a good 2 BHK Apartment in Tilak Na...,3,Yes,True,New Delhi,Krishna Park Extension,4000000.0,2,BHK,Flat
14524,"Rawta, Jaffarpur Kalan,New Delhi",1 BHK Independent House for sale in Jaffarpur ...,₹14.0 L,"Rawta, Jaffarpur Kalan,New Delhi",472,2970.0,1 BHK Independent House for sale in Jaffarpur ...,2,Yes,True,New Delhi,Rawta,1400000.0,1,BHK,Independent House
14525,"Rani Garden, Geeta Colony,New Delhi","1 BHK Flat for sale in Geeta Colony, New Delhi",₹30.0 L,"Rani Garden, Geeta Colony,New Delhi",378,7940.0,"Property for sale in Geeta Colony, Delhi. This...",1,Yes,True,New Delhi,Rani Garden,3000000.0,1,BHK,Flat
14526,"Lig flat rohini,Sector 16E, Sector 16 Rohini,N...","3 BHK Flat for sale in Sector 16 Rohini, New D...",₹60.0 L,"Lig flat rohini,Sector 16E, Sector 16 Rohini,N...",700,8570.0,Check out this 3 BHK Apartment for sale in Roh...,2,Yes,True,New Delhi,Lig flat rohini,6000000.0,3,BHK,Flat


In [55]:
print(final["Price_per_SQFT"].min(), final["Price_per_SQFT"].max())
print(final["price_parsed"].min(), final["price_parsed"].max())

0.0 999000.0
55000.0 840000000.0


In [56]:
final = final.drop([
        "Property Title",
        "Price",
        "Location",
        "Balcony"
], axis=1) \
.astype(
    {
        "city":           "category",   # Category data types are much more efficient compared to objects or String
        "suburb":         "category",
        "Bed Rooms":      "int8",       # Don't think people live in houses with more than 255 bedrooms or bathrooms
        "Baths" :         "int8",
        "Room Type":      "category",
        "Property Type":  "category",
        "Description":    "string",
        "Name" :          "string",
        "Price_per_SQFT": "float32",
        "price_parsed" :  "int32"
    }
).rename({
    "price_parsed" : "price"
}, axis=1)

In [57]:

final.dtypes

Name                string
Total_Area           int64
Price_per_SQFT     float32
Description         string
Baths                 int8
hasBalcony            bool
city              category
suburb            category
price                int32
Bed Rooms             int8
Room Type         category
Property Type     category
dtype: object

In [48]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14528 entries, 0 to 14527
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Name            14528 non-null  string  
 1   Total_Area      14528 non-null  int64   
 2   Price_per_SQFT  14528 non-null  float32 
 3   Description     14528 non-null  string  
 4   Baths           14528 non-null  int8    
 5   hasBalcony      14528 non-null  bool    
 6   city            14528 non-null  category
 7   suburb          14528 non-null  category
 8   price_parsed    14528 non-null  int32   
 9   Bed Rooms       14528 non-null  int8    
 10  Room Type       14528 non-null  category
 11  Property Type   14528 non-null  category
dtypes: bool(1), category(4), float32(1), int32(1), int64(1), int8(2), string(2)
memory usage: 877.4 KB


From this we can see that we have managed to decrease the memory usage by 8.5% across the 14000+ Rows and despite making additional columns

In [58]:
final.to_excel(DATA_DIR / "clean" / "Real_Estate_Cleaned.xlsx", index=False)